In [1]:
%load_ext autoreload
%autoreload 2

import warnings
import pandas as pd
import numpy as np
import os
import sys # error msg, add the modules
import operator # sorting
from math import *
import matplotlib.pyplot as plt

sys.path.append('../../../')

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

import prep
from warp import *

In [2]:
trace_file = 'trace.csv'
metrics_file = 'metrics.csv'

# read trace and metrics into dataframe, from generated trace file
df_kernel_trace, df_metrics = prep.Prep_trace_metrics(trace_file, metrics_file)

In [3]:
unique_kerns = []
for index, kenname in enumerate(df_kernel_trace.Name.unique()):
    if index > 0 and 'memcpy' not in kenname:
        head = kenname.split('(')[0]
        if '<' in head:
            head = head.split('<')[0]
        #print head
    
        if 'void' in head:
            head = head.split(' ')
            #print head[1]
            head = head[1]
            
        if head not in unique_kerns:
            unique_kerns.append(head)
            print head

mcx_main_loop


In [4]:
target_kern_name = 'mcx_main_loop'

sass_result = target_kern_name + '.sm_52.sass.result'
kernel_stat_out ='kernelStat_' + target_kern_name + '.csv'

out_columns = prep.init_df_columns(df_metrics)
df_kern = pd.DataFrame(columns=out_columns)

# current kernel trace
df_current = prep.GenCurKernInfo(df_kernel_trace, df_metrics,target_kern_name, out_columns)
# add current trace
df_kern = df_kern.append(df_current, ignore_index=True)

warp_inst = WarpInst()
warp_inst.run(df_kern, sass_result)

df_kern.insert(1, 'm2c_ratio', warp_inst.mem_clks / warp_inst.cmp_clks)
df_kern.to_csv(kernel_stat_out, index=False, encoding='utf-8')
df_kern

LDG (global load) clocks (per warp) 		 = 10632700.0
STG (global store) clocks (per warp) 		 = 57.0
LDS (shared memory load) clocks (per warp) 	 = 404378.0
STS (shared memory store) clocks (per warp) 	 = 119225.0
=> Memory inst. (per warp) 			 = 11156360.0 (clocks)

Integer inst. (per warp) 		 = 17139240.0 (clocks)
FP32 inst. (per warp) 			 = 21513795.0 (clocks)
FP64 inst. (per warp) 			 = 955008.0 (clocks)
Compute load store inst. (per warp) 	 = 2482380.0 (clocks)
=> Compute inst. (per warp) 		 = 42090423.0 (clocks)

Compute Intensive : cmp / mem = 3.77277382587


,kern_name,m2c_ratio,duration_ms,gridx,gridy,gridz,blkx,blky,blkz,reg_per_thread,...,l2_utilization,l2_atomic_throughput,l2_atomic_transactions,sysmem_utilization,ecc_throughput,sysmem_read_utilization,sysmem_write_utilization,eligible_warps_per_cycle,flop_sp_efficiency,flop_dp_efficiency
0,mcx_main_loop,0.265057,188.351033,192.0,1.0,1.0,64.0,1.0,1.0,68.0,...,Low (3),23.194589GB/s,242580238,Low (1),0.000000B/s,Low (1),Low (1),5.135198,3.662662%,0.000000%
